## Importing libraries

In [1]:
!pip install xgboost

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## XGBoost implementation

### Preparing the Dataset

In [23]:
all_prices = pd.read_csv('/content/drive/MyDrive/ml-datasets/currencies_prediction/prices_30m.csv')

In [24]:
# adding features that make more sense than raw data
all_prices["Gold_ret"] = all_prices["Gold"].pct_change()
all_prices["Silver_ret"] = all_prices["Silver"].pct_change()
all_prices["CAD_ret"] = all_prices["CAD"].pct_change()

all_prices["Gold_ma"] = all_prices["Gold"].rolling(5).mean()
all_prices["Silver_ma"] = all_prices["Silver"].rolling(5).mean()
all_prices["CAD_ma"] = all_prices["CAD"].rolling(5).mean()
all_prices = all_prices.dropna()


In [ ]:
# Lag to simulate previous days/windows
for lag in range(1, 31):
  all_prices[f"Gold_lag_{lag}"] = all_prices["Gold"].shift(lag)
  all_prices[f"Silver_lag_{lag}"] = all_prices["Silver"].shift(lag)
  all_prices[f"CAD_lag_{lag}"] = all_prices["CAD"].shift(lag)

In [31]:
# price to predict, the price of the next timestep
all_prices["Gold_target"] = all_prices["Gold"].shift(-1)
all_prices["Silver_target"] = all_prices["Silver"].shift(-1)
all_prices["CAD_target"] = all_prices["CAD"].shift(-1)

In [32]:
all_prices = all_prices.dropna()

In [27]:
all_prices.head()

,Datetime,Gold,Silver,CAD,Gold_ret,Silver_ret,CAD_ret,Gold_ma,Silver_ma,CAD_ma,...,CAD_lag_28,Gold_lag_29,Silver_lag_29,CAD_lag_29,Gold_lag_30,Silver_lag_30,CAD_lag_30,Gold_target,Silver_target,CAD_target
34,2025-09-09 16:00:00+00:00,3726.300049,43.514999,0.722554,0.0,0.0,-0.000383,3726.300049,43.514999,0.723125,...,0.724638,3726.300049,43.514999,0.724475,3726.300049,43.514999,0.724071,3726.300049,43.514999,0.722966
35,2025-09-09 16:30:00+00:00,3726.300049,43.514999,0.722966,0.0,0.0,0.000571,3726.300049,43.514999,0.722976,...,0.724533,3726.300049,43.514999,0.724638,3726.300049,43.514999,0.724475,3726.300049,43.514999,0.722575
36,2025-09-09 17:00:00+00:00,3726.300049,43.514999,0.722575,0.0,0.0,-0.000542,3726.300049,43.514999,0.722780,...,0.724255,3726.300049,43.514999,0.724533,3726.300049,43.514999,0.724638,3726.300049,43.514999,0.722361
37,2025-09-09 17:30:00+00:00,3726.300049,43.514999,0.722361,0.0,0.0,-0.000296,3726.300049,43.514999,0.722657,...,0.724144,3726.300049,43.514999,0.724255,3726.300049,43.514999,0.724533,3726.300049,43.514999,0.722262
38,2025-09-09 18:00:00+00:00,3726.300049,43.514999,0.722262,0.0,0.0,-0.000137,3726.300049,43.514999,0.722543,...,0.724286,3726.300049,43.514999,0.724144,3726.300049,43.514999,0.724255,3726.300049,43.514999,0.722189


In [28]:
all_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2784 entries, 34 to 2817
Columns: 103 entries, Datetime to CAD_target
dtypes: float64(102), object(1)
memory usage: 2.2+ MB


In [29]:
all_prices.describe()

,Gold,Silver,CAD,Gold_ret,Silver_ret,CAD_ret,Gold_ma,Silver_ma,CAD_ma,Gold_lag_1,...,CAD_lag_28,Gold_lag_29,Silver_lag_29,CAD_lag_29,Gold_lag_30,Silver_lag_30,CAD_lag_30,Gold_target,Silver_target,CAD_target
count,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,...,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000,2784.000000
mean,3999.327739,48.253049,0.715633,0.000049,0.000103,-0.000004,3998.950512,48.243304,0.715638,3999.139125,...,0.715730,3994.205828,48.130860,0.715735,3994.041891,48.127315,0.715739,3999.516352,48.257922,0.715631
std,171.183478,2.998239,0.004810,0.002063,0.003421,0.000373,171.043356,2.987622,0.004809,171.194923,...,0.004882,171.887219,2.944219,0.004885,171.925020,2.943833,0.004887,171.171825,3.001557,0.004809
min,3721.399902,43.514999,0.707269,-0.014739,-0.039054,-0.002440,3724.160010,43.514999,0.707599,3721.399902,...,0.707269,3721.399902,43.514999,0.707269,3721.399902,43.514999,0.707269,3721.399902,43.514999,0.707269
25%,3865.300049,46.773751,0.712400,-0.000624,-0.001053,-0.000200,3864.775037,46.745000,0.712391,3863.650024,...,0.712403,3851.725037,46.605000,0.712403,3851.375000,46.605000,0.712411,3866.175049,46.775002,0.712400
50%,4018.349976,48.230000,0.714411,0.000000,0.000000,-0.000007,4017.750000,48.225000,0.714415,4018.250000,...,0.714439,4015.250000,48.180000,0.714441,4015.049927,48.180000,0.714446,4018.500000,48.230000,0.714411
75%,4126.625122,50.586249,0.717907,0.000898,0.001473,0.000194,4127.195093,50.583250,0.717930,4126.600098,...,0.718181,4123.175049,50.529999,0.718184,4122.949951,50.526251,0.718184,4126.800171,50.590000,0.717897
max,4394.299805,57.080002,0.728014,0.014620,0.045237,0.002382,4383.499902,57.080002,0.727883,4394.299805,...,0.728014,4394.299805,54.299999,0.728014,4394.299805,54.299999,0.728014,4394.299805,57.080002,0.728014
